In [ ]:
#import nltk and corpus
import nltk
from nltk.corpus import brown

#import numpy and pandas
import numpy as np
import pandas as pd

#read the bible and make a frequency plot of the most frequent words
nltk.download('gutenberg')
nltk.download('stopwords')

bible = nltk.corpus.gutenberg.words('bible-kjv.txt')
fdist = nltk.FreqDist(w.lower() for w in bible)

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from string import punctuation

#read the bible and make a frequency plot of the most frequent words
bible = nltk.corpus.gutenberg.words('bible-kjv.txt')
fdist = nltk.FreqDist(w.lower() for w in bible)

# make a list of the most frequent words excluding the stopwords
most_freq_words = [w for w in fdist.keys() if w not in nltk.corpus.stopwords.words('english') and not w.isdigit()]
most_freq_words = [w for w in most_freq_words if w not in punctuation]

# make a dataframe of the most frequent words
df = pd.DataFrame(most_freq_words, columns=['word'])

# add a column for the frequency
df['freq'] = df['word'].map(fdist)
# add a column for the percentage
df['percent'] = df['freq']/df['freq'].sum()
# add a column for the rank
df['rank'] = df['freq'].rank(ascending=False)
# add a column for the cumulative frequency
df['cum_freq'] = df['freq'].cumsum()
# add a column for the cumulative percentage
df['cum_percent'] = df['cum_freq']/df['freq'].sum()
# add a column for the log rank
df['log_rank'] = np.log(df['rank'])

In [ ]:
df.sort_values(by='rank', ascending=True)[:30]

,word,freq,percent,rank,cum_freq,cum_percent,log_rank
118,shall,9838,0.026224,1.0,78463,0.209149,0.000000
42,unto,8997,0.023982,2.0,37277,0.099365,0.693147
133,lord,7964,0.021229,3.0,88838,0.236805,1.098612
189,thou,5474,0.014591,4.0,105123,0.280214,1.386294
259,thy,4600,0.012262,5.0,131649,0.350921,1.609438
11,god,4472,0.011920,6.0,9713,0.025891,1.791759
25,said,3999,0.010660,7.0,20059,0.053469,1.945910
232,ye,3983,0.010617,8.0,121246,0.323191,2.079442
262,thee,3827,0.010201,9.0,135951,0.362388,2.197225
19,upon,2748,0.007325,10.0,14712,0.039216,2.302585


In [ ]:
def markov_chain(text):
    """The input is a string of text and the output will be a dictionary with each word as
       a key and each value as the list of words that come after the key in the text."""
    
    # Tokenize the text by word if text is a list and if text is not a list
    # split by space
    list_text = text if type(text) == list else text.split(" ")
    
    # Create an empty dictionary to hold the word pairs
    word_dict = {}
    
    # Create a list of tuples by pairing the words with the words that follow them
    for current_word, next_word in zip(list_text, list_text[1:]):
        
        # Create a new key value pair if current_word is not already a key in the word_dict
        if current_word not in word_dict.keys():
            word_dict[current_word] = [next_word]
        
        # Add the next word to the list of values for the current word key in the word_dict
        else:
            word_dict[current_word].append(next_word)
    
    # return the dictionary
    return word_dict

In [ ]:

bible_list = list(bible)

# Create the dictionary for the word "I"
bible_dict = markov_chain(bible_list)

# Create a function that generates sentences
def generate_sentence(chain, count=15):
    '''Input a dictionary in the format of key = current word, value = list of next words
       along with the number of words you would like to see in your generated sentence.'''

    # Capitalize the first word
    word1 = np.random.choice(list(chain.keys()))
    sentence = word1.capitalize()

    # Generate the second word from the value list. Set the new word as the first word. Repeat.
    for i in range(count-1):
        word2 = np.random.choice(chain[word1])
        word1 = word2
        sentence += ' ' + word2

    # End it with a period
    sentence += '.'
    return(sentence)

# Generate sentences from the bible
for i in range(3):
    print(generate_sentence(bible_dict))

# Create a function to generate a paragraph
def generate_paragraph(chain, count=4, sent_count=4):
    '''Input a dictionary in the format of key = current word, value = list of next words
       along with the number of words you would like to see in your generated paragraph.'''
    
    # Capitalize the first word
    word1 = np.random.choice(list(chain.keys()))
    paragraph = word1.capitalize()

    # Generate the second word from the value list. Set the new word as the first word. Repeat.
    for i in range(count-1):
        word2 = np.random.choice(chain[word1])
        word1 = word2
        paragraph += ' ' + word2

    # End it with a period
    paragraph += '.'
    
    # Generate sentences from the paragraph
    for i in range(sent_count):
        paragraph += ' ' + generate_sentence(chain)
    
    return(paragraph)

# Generate paragraphs from the bible
for i in range(10):
    print(generate_paragraph(bible_dict))

Loaves : 8 : 23 Am I command them that doeth good conversation received :.
Asketh of fire unto all the LORD hath filled with death ; but woe also.
Preservest man should not strawed : 11 Again , to the guard ; that thou.
Licked up : 15. Geba ; they shouted with a man , to let it for the LORD .. Fragments that came and all those husbandmen of thy mother and entered into the east. Screech owl , I will I will I pray for a great unto one that. Firebrand plucked away the eye shall have I shall be of Beor to abide over.
Adjured the host of. Hatita , to mourn , Go ye gave her , and , 10 All the. Hupham , how shall measure , which provoketh to the truth of David was a. Freewill offering , because thou , his house of hewn stone to pass by might. Appearing of men of the LORD . 2 : and when the LORD God :.
Galley with fir trees. Seated ; now do these things .) 8 : but of Canaan , and he. Benefactors . 28 : 17 And thou answer us not do justice and spoiled :. Kithlish , and Kedesh in the heathen , a

In [ ]:
# Here's a LSTM model to generate bible passages
import keras
import numpy as np

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

# Load in the bible text
# load ascii text and covert to lowercase
def listToString(s):   
    str1 = " "    
    return (str1.join(s))

raw_text = listToString(list(bible)).lower()
#filename = "bible-kjv.txt"
#raw_text = open(filename).read()
#raw_text = raw_text.lower()

# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)



Total Characters:  4495207
Total Vocab:  49


In [8]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)


Total Patterns:  4495107


In [9]:

# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')



In [11]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.fit(X, y, epochs=20, batch_size=1024*2, callbacks=callbacks_list)


Epoch 1/20
2195/2195 [==============================] - 191s 85ms/step - loss: 1.2689

Epoch 00001: loss improved from inf to 1.26894, saving model to weights-improvement-01-1.2689.hdf5
Epoch 2/20
2195/2195 [==============================] - 186s 85ms/step - loss: 1.2601

Epoch 00002: loss improved from 1.26894 to 1.26011, saving model to weights-improvement-02-1.2601.hdf5
Epoch 3/20
2195/2195 [==============================] - 186s 85ms/step - loss: 1.2561

Epoch 00003: loss improved from 1.26011 to 1.25612, saving model to weights-improvement-03-1.2561.hdf5
Epoch 4/20
2195/2195 [==============================] - 187s 85ms/step - loss: 1.2530

Epoch 00004: loss improved from 1.25612 to 1.25297, saving model to weights-improvement-04-1.2530.hdf5
Epoch 5/20
1007/2195 [============>.................] - ETA: 1:40 - loss: 1.2486

KeyboardInterrupt: ignored

In [ ]:

# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters
for i in range(300):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    print(result, end='')
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

In [47]:
import random
p = model.predict(x, verbose=0)
p.argmax)


27

In [50]:
def top_argmax(arr, n):
    return np.random.choice(np.argsort(arr)[-3:])


In [12]:
# Generate bible verse
# load the network weights
filename = "weights-improvement-04-1.2530.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')